In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import numpy as np

In [2]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

In [3]:
#업로더 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

#업로더 홈페이지 접속
url = 'https://www.youtube.com/c/%ED%95%8F%EB%8D%94%EC%82%AC%EC%9D%B4%EC%A6%88/videos'
driver.get(url)

#driver.maximize_window()

In [4]:
#카테고리 리스트
keyword_list = ['조명', '무드등', '형광등', '방등', '거실등', \
    '침구', '매트리스', '블라인드', '커튼', '러그', '매트', '쿠션', '소파', \
    '테이블', '식탁', '책상', '거실장', '서랍', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '거울']

In [10]:
#키워드 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
keyword = '조명 추천'

url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)

#driver.maximize_window()

# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)

# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

# 업로드 날짜 -> 올해 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

In [11]:
# 제목 & 링크 크롤링

# 스크롤 다운 10번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,10):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))
    
youtubeDic = {
    '영상 번호' : video_num_list,
    '제목': name_list,
    '주소': url_list,
}

youtubeDf = pd.DataFrame(youtubeDic)
youtubeDf.head()

,영상 번호,제목,주소
0,1,조명추천💡조명만 몇백 개 본 리빙 편집숍 디자인팀이 추천하는 조명,https://www.youtube.com/watch?v=Bz0PEb5CSTo
1,2,[소장각💡조명 추천] 이케아에서 꼭 사야하는 조명 3가지!!,https://www.youtube.com/watch?v=EcW4py8igtg
2,3,"인테리어 조명 추천 8 | 루이스폴센 판텔라, 루이스폴센 PH 4/3, 아르떼미데 ...",https://www.youtube.com/watch?v=N35VRUbu6_E
3,4,"내돈내산 1만원~90만원대 인테리어 조명 추천 5가지 | 루이스폴센, 우메이, 이케...",https://www.youtube.com/watch?v=bAgtZteiQAw
4,5,"우리집 분위기 담당✨ 인테리어 조명 추천 & 리뷰 | 렉슨, 일광전구, 이케아, 선...",https://www.youtube.com/watch?v=EP8tbIzeBFs


In [16]:
youtubeDf.to_csv('C:\jupiter_workspace\zam_project\outputs/유튜브_'+keyword + '.csv', encoding='utf-8-sig', index=False)

# #############여기까지만#############

In [12]:
# 댓글 크롤링
video_num = 0
video_idex=[]
video_comm = []


for i in range(len(name_list)):
    driver.get(url_list[i])
    
    time.sleep(3)
    
    try :
        # 동영상 재생 멈추기 
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.SPACE)
    except: # 스페이스 누르기 안되면 직접 일시중지 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[1]/video').click()
        
    # 유튜브 프리미엄 팝업창 닫기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass

    # 댓글 보이게 하기(밑으로 내리기)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,800);")
    time.sleep(8)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(5)
    
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        temp_comment = temp_comment.replace('\xa0@', '')
        temp_comment = temp_comment.replace('\xa0', '')
        temp_comment = temp_comment.replace('\r', '')
        comment_final.append(temp_comment)
    
    video_idex.append(video_num)
    video_comm.append(comment_final)
    video_num += 1


KeyboardInterrupt: 

In [23]:
df = []
for i in range(len(video_idex)):
    if len(video_comm[i]) != 0:
        n = np.full((len(video_comm[i])),i+1)
        temp = pd.DataFrame([ x for x in zip(n,video_comm[i])])
        df.append(temp)

In [24]:
comment_df = df[0]
for i in range(1, len(df)):
    comment_df = pd.concat([comment_df, df[i]])
    
comment_df.columns = ['영상 번호', '댓글']
comment_df.head()

,영상 번호,댓글
0,1,이런 컨텐츠 좋은 거 같아요~몰랐던 브래드도 알고 메인템도 알아서 좋네요~
1,1,각 브랜드에서 매력있는 아이템들이 많은걸 알아서 좋았네요 ㅎㅎ
2,1,형 이런 컨셉 유료광고면 한달에 3번 이상 유료광고 받아도 볼만한 느낌… 짱이다 형️
3,1,마지막 바지 예뻐서 사려고봤더니 품절이네요 ㅠㅠ 재입고되면 바로사야지 !
4,1,와 데님셔츠는 가격대비 미치긴한듯


In [11]:
#CSV형태로 저장하기
youtubeDf.to_csv('./'+'유튜브_핏더사이즈_크롤링.csv', encoding='utf-8-sig', index=False)
comment_df.to_csv('./'+'유튜브_핏더사이즈_댓글.csv', encoding='utf-8-sig', index=False)